In [1]:
import pandas as pd
import numpy as np
import theano.tensor as T
import theano
import theano.gradient
import pyipopt

In [2]:
inputfile = './data_new_volume.csv'
df = pd.read_csv(inputfile)
df['const'] = 1

df['pgmidgrade_km_adj'].fillna(value=10000, inplace=True)

df = df[(df['treattype'] < 3)]
for elem in df['treattype'].unique():
    df['treat' + str(elem)] = df['treattype'] == elem

for elem in df['choice'].unique():
    df['choice' + str(elem)] = df['choice'] == elem

df['treat1_topusage'] = df['treat1']*df['dv_usageveh_p75p100']
#df['treat2_topusage'] = df['treat2']*df['dv_usageveh_p75p100']

df['treat1_college'] = df['treat1']*df['dv_somecollege']
#df['treat2_college'] = df['treat2']*df['dv_somecollege']

df.head()

In [3]:
#df = df[df['choice']!=2]
#df['choice'][df['choice']==3] = 2

In [4]:
#df = df[ df['dv_carpriceadj_p75p100'] == 1]
#df = df[ df['dv_usageveh_p75p100'] == 0]
df.shape

In [5]:
price_labels = ['pg_km_adj', 'pgmidgrade_km_adj', 'pe_km_adj']
value_labels = ['value_total']

# all
Xexpd_labels = ['choice2', 'choice3', 'dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# usage
#Xexpd_labels = ['choice2', 'choice3', 'dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_carpriceadj_p75p100', 'stationvisit_avgcarprice_adj', 'const']

# car price
#Xexpd_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']
#Xutil_labels = ['dv_ctb', 'dv_bh', 'dv_rec', 'dv_female', 'dv_age_25to40y', 'dv_age_morethan65y', 'dv_somesecondary', 'dv_somecollege', 'dv_usageveh_p75p100', 'stationvisit_avgcarprice_adj', 'const']

Xalpha_labels = ['const']
Xlsigma_labels = ['treat1', 'treat2']

choice = df.loc[:, 'choice'].as_matrix()
price = df.loc[:, price_labels].as_matrix()
value = np.squeeze(df.loc[:, value_labels].as_matrix())
Xexpd = df.loc[:, Xexpd_labels].as_matrix()
Xutil = df.loc[:, Xutil_labels].as_matrix()

Xalpha = df.loc[:, Xalpha_labels].as_matrix().astype(int)
Xlsigma = df.loc[:, Xlsigma_labels].as_matrix()

nobs, nchoice = price.shape
nobs, nXutil = Xutil.shape

nobs, nXalpha = Xalpha.shape
nobs, nXlsigma = Xlsigma.shape

choiceidx = np.ravel_multi_index((range(nobs), choice-1), (nobs, nchoice))

pchoice = price[range(nobs), choice-1]

ndraws = 500

In [6]:
# construct the look-up table to quickly find the actual choice for each individual
# i.e. given index (i,r), find index (i,j,r) with j = choice of i
ii, rr = np.mgrid[0:nobs, 0:ndraws]
jj = np.tile(choice.reshape(nobs,1)-1, (1,ndraws))
choice_flat_idx_3D = np.ravel_multi_index((ii, jj, rr), (nobs, nchoice, ndraws))


In [7]:
np.random.seed(1234)
draws = np.random.normal(0., 1., (nobs, nchoice, ndraws))
draws = draws - np.sum(draws, axis=2, keepdims=True)/ndraws

$u_{ijr} = \alpha_i p_j + X_i \beta_j + \rho_j z_{ir} + \mu\epsilon_{ij} = V_{ij} + \rho_j z_{ir} + \mu_i\epsilon_{ij}$

$z_{ir}$ is used to capture the correlation in between choices. $\rho_0 = 0, \rho_1 = \rho_2 = \rho$ if positive correlation and $ \rho_1 = -\rho_2 = \rho$ if negative correlation

$P(ijr) = \frac{\exp\left(\frac{V_{ij} + \rho_j z_{ir}}{\mu_i}\right)}{\sum_k \exp\left(\frac{V_{ik} + \rho_k z_{ir}}{\mu_i}\right)}$

$P(ij) = \frac{1}{R} \sum_r \frac{\exp\left(\frac{V_{ij} + \rho_j z_{ir}}{\mu_i}\right)}{\sum_k \exp\left(\frac{V_{ik} + \rho_k z_{ir}}{\mu_i}\right)} =  \frac{1}{R} \exp\left(\frac{V_{ij}}{\mu_i}\right) \sum_r \frac{\exp\left(\frac{\rho_j z_{ir}}{\mu_i}\right)}{\sum_k \exp\left(\frac{V_{ik} + \rho_k z_{ir}}{\mu_i}\right)}$

$\ln P(ij) = -\ln R + \frac{V_{ij}}{\mu_i} + \ln \left(\ \sum_r \frac{\exp\left(\frac{\rho_j z_{ir}}{\mu_i}\right)}{\sum_k \exp\left(\frac{V_{ik} + \rho_k z_{ir}}{\mu_i}\right)} \right)$

In [10]:
floatX = 'float64'

theta0 = np.zeros(nXalpha + nXlsigma + (nchoice-1)*nXutil)
theta0[0] = -20

def getparams(theta):
    offset = 0
    gammaalpha = theta[offset:offset+nXalpha].reshape((nXalpha, 1))
    
    offset += nXalpha
    gammalsigma = theta[offset:offset+nXlsigma].reshape((nXlsigma, 1))

    offset += nXlsigma
    betautil = theta[offset:offset+(nchoice-1)*nXutil].reshape((nXutil, nchoice-1))
        
    return gammaalpha, gammalsigma, betautil

theta = T.dvector('theta')
gammaalpha, gammalsigma, betautil = getparams(theta)

priceT = theano.shared(price.astype(floatX), name='price')
pchoiceT = theano.shared(pchoice.astype(floatX), name='pchoice')
XutilT = theano.shared(Xutil.astype(floatX), name='Xutil')
XalphaT = theano.shared(Xalpha.astype(floatX), name='Xalpha')
XlsigmaT = theano.shared(Xlsigma.astype(floatX), name='Xlsigma')
drawsT = theano.shared(draws, name = 'draws')

sigma = T.exp(T.dot(XlsigmaT, gammalsigma))
alpha = T.dot(Xalpha, gammaalpha)
mu = 2

util = alpha*priceT + T.concatenate([T.zeros((nobs, 1), dtype=floatX), T.dot(XutilT, betautil)], axis=1)
util_noise = sigma.dimshuffle(0,1,'x')*drawsT
util2 = util.dimshuffle((0,1,'x')) + util_noise - util_noise.flatten()[choice_flat_idx_3D].reshape((nobs,1,ndraws))
nlogl = nobs*np.log(ndraws) - T.log(T.sum(1/T.sum(T.exp(util2/mu), axis=1), axis=1)).sum() - util.flatten()[choiceidx].sum()/mu 

In [11]:
theano.function([theta], outputs = nlogl)(theta0)

In [12]:
def buildfunc(theta, nloglf):
    return (theano.function([theta], outputs = nloglf),
        theano.function([theta], outputs = T.grad(nloglf, [theta])),
        theano.function([theta], outputs = theano.gradient.hessian(nloglf, [theta])))

def buildeval(theta, nloglf):
    f, grad, hess = buildfunc(theta, nloglf)
    
    def eval_f(thetavalue):
        return f(thetavalue)

    def eval_grad(thetavalue):
        return np.squeeze(grad(thetavalue))

    def eval_hess(thetavalue):
        return np.squeeze(hess(thetavalue))

    return eval_f, eval_grad, eval_hess

eval_f, eval_grad, eval_hess = buildeval(theta, nlogl)

In [ ]:
#eval_hess(theta0)

In [ ]:
pyipopt.set_loglevel(1)
thetahat , _, _, _, _, fval = pyipopt.fmin_unconstrained(
    eval_f,
    theta0,
    fprime=eval_grad,
    fhess=eval_hess,
    )


In [ ]:
covhat = np.linalg.inv(eval_hess(thetahat))
se = np.sqrt(np.diagonal(covhat))
t = thetahat/se

gammaalphahat, gammalsigmahat, betautilhat = getparams(thetahat)
gammaalphase, gammalsigmase, betautilse = getparams(se)
gammaalphat, gammalsigmat, betautilt = getparams(t)

formatstr = '%30s%10.3f%10.3f%10.3f'

print '-'*60

print ' \n*** alpha equation'
for i in range(nXalpha):
    print formatstr % (Xalpha_labels[i], gammaalphahat[i], gammaalphase[i], gammaalphat[i])
print '-'*60

print ' \n*** ln(sigma) equation'
for i in range(nXlsigma):
    print formatstr % (Xlsigma_labels[i], gammalsigmahat[i], gammalsigmase[i], gammalsigmat[i])
print '-'*60

print ' \n*** Discrete choice equation'
for j in range(nchoice-1):
    print '-------- choice', j+1, '------------------------------------------'
    for i in range(nXutil):
        print formatstr % (Xutil_labels[i], betautilhat[i][j], betautilse[i][j], betautilt[i][j])
print '-'*60


In [ ]:

sigmahat = np.exp(gammalsigmahat)

1-(np.pi**2/6*mu**2 + sigmahat**2)/(np.pi**2/6*mu**2 + 1)